
# Moving Average Strategy Backtesting

This Jupyter Notebook demonstrates a simple moving average strategy for backtesting stock data. The strategy involves calculating moving averages for a given stock and generating buy/sell signals based on the relationship between the stock's adjusted close price and its moving average.

## Steps:

1. **Import Libraries**: Import necessary libraries such as `numpy`, `pandas`, `matplotlib.pyplot`, and `yfinance`.

2. **Download Stock Data**: Use the `yfinance` library to download historical stock data for a specified ticker symbol (e.g., "GOOG") and date range.

3. **Calculate Moving Averages**: Define a function to calculate the moving average of the stock's adjusted close price over a specified window.

4. **Strategy Implementation**: Define a function to implement the moving average strategy. This function calculates the moving average, generates buy/sell signals, and computes the strategy's returns.

5. **Backtesting the Strategy**: Define a function to backtest the strategy by comparing the strategy's returns with a buy-and-hold approach.

6. **Test Different Moving Averages**: Test the strategy with different moving average windows and store the results.

7. **Plot Results**: Plot the cumulative returns of the moving average strategy and compare them with the buy-and-hold returns.

## Variables:

- `ticker`: The stock ticker symbol (e.g., "GOOG").
- `start_date`: The start date for downloading historical stock data.
- `end_date`: The end date for downloading historical stock data.
- `data`: The downloaded stock data.
- `moving_average_windows`: A list of different moving average windows to test.
- `results`: A dictionary to store the backtested data for each moving average window.
- `backtested_data`: The final backtested data for the last moving average window tested.

## Functions:

- `calculate_moving_average(data, window)`: Calculates the moving average for the given data and window.
- `moving_average_strategy(data, ma_window)`: Implements the moving average strategy and returns the data with strategy signals and returns.
- `backtest_strategy(data)`: Backtests the strategy by calculating cumulative returns for both the strategy and a buy-and-hold approach.

## Usage:

Run the notebook cells sequentially to download the stock data, calculate moving averages, implement the strategy, backtest it, and plot the results.

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

# Download Tesla stock data
ticker = "GOOG"
start_date = "2020-01-01"
end_date = "2024-01-01"
data = yf.download(ticker, start=start_date, end=end_date)

# Calculate moving averages
def calculate_moving_average(data, window):
    return data['Adj Close'].rolling(window=window).mean()

# Strategy implementation
def moving_average_strategy(data, ma_window):
    data['MA'] = calculate_moving_average(data, ma_window)
    data['Signal'] = 0
    data['Signal'][ma_window:] = np.where(data['MA'][ma_window:] > data['Adj Close'][ma_window:], 1, -1)
    data['Position'] = data['Signal'].shift(1)  # Shift to avoid lookahead bias
    data['Strategy Returns'] = data['Position'] * data['Adj Close'].pct_change()
    return data

# Backtesting the strategy
def backtest_strategy(data):
    # Calculate Buy and Hold returns
    data['Buy and Hold Returns'] = data['Adj Close'].pct_change().cumsum()
    
    # Calculate Strategy returns
    data['Cumulative Strategy Returns'] = data['Strategy Returns'].cumsum()
    
    return data

# Test the strategy with different moving averages
moving_average_windows = [5,10,20,20,40, 50, 100]
results = {}

for window in moving_average_windows:
    strategy_data = moving_average_strategy(data.copy(), window)
    backtested_data = backtest_strategy(strategy_data)
    results[window] = backtested_data

# Plotting the results
plt.figure(figsize=(14, 7))
print(f"Buy and Hold Returns: {backtested_data['Buy and Hold Returns'][-1]}")
for window, result in results.items():
    print(f"Strategy Returns: {result['Cumulative Strategy Returns'][-1]}")
for window, result in results.items():
    plt.plot(result['Cumulative Strategy Returns'], label=f'Strategy MA {window} Days')
    
plt.plot(result['Buy and Hold Returns'], label='Buy and Hold', linestyle='--')
plt.title('Cumulative Returns: Moving Average Strategy vs Buy and Hold')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.show()